In [1]:
import pandas as pd
import funciones as f
df=pd.read_csv(r"zalando\data\datos_aleatorios_2023.csv")

df=f.funcion_principal(df)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 928 entries, 0 to 999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           928 non-null    datetime64[ns]
 1   time           928 non-null    object        
 2   model          928 non-null    object        
 3   color          928 non-null    object        
 4   current_price  928 non-null    float64       
 5   normal_price   928 non-null    float64       
 6   link           928 non-null    object        
 7   year           928 non-null    int32         
 8   month          928 non-null    int32         
 9   marcas         928 non-null    object        
dtypes: datetime64[ns](1), float64(2), int32(2), object(5)
memory usage: 72.5+ KB


In [3]:
df.sample(100)

,date,time,model,color,current_price,normal_price,link,year,month,marcas
643,2023-08-25,08:58:21,Nano,black/midnight plum/coral,114.83,143.78,https://www.zalando.es/reebok-nano-x4-zapatill...,2023,8,Reebok
15,2023-01-11,10:27:11,Tribase Reign 6,white/distant gray/black,136.82,148.15,https://www.zalando.es/under-armour-tribase-re...,2023,1,Under Armour
426,2023-06-10,02:28:21,Tribase Reign 6,white/distant gray/black,146.54,143.87,https://www.zalando.es/under-armour-tribase-re...,2023,6,Under Armour
914,2023-12-01,18:37:57,Metcon 9,light silver/metallic silver/summit white/jade...,121.15,152.20,https://www.zalando.es/nike-performance-metcon...,2023,12,Nike
478,2023-06-29,18:13:26,Nano,hyper green black vector red,81.11,83.75,https://www.zalando.es/reebok-nano-x4-zapatill...,2023,6,Reebok
...,...,...,...,...,...,...,...,...,...,...
896,2023-11-23,08:09:56,Nano,footwear white/black,121.76,144.57,https://www.zalando.es/reebok-nano-x4-zapatill...,2023,11,Reebok
245,2023-04-05,09:56:12,Metcon 9,smokey mauve/black/platinum violet,132.47,150.43,https://www.zalando.es/nike-performance-metcon...,2023,4,Nike
605,2023-08-11,22:05:29,Nfx Trainer,white,69.59,70.39,https://www.zalando.es/reebok-nfx-trainer-zapa...,2023,8,Reebok
834,2023-10-31,21:26:59,Fuse 3.0,black/redmazing,134.14,132.80,https://www.zalando.es/puma-fuse-30-zapatillas...,2023,10,Puma


In [4]:
df_nike = df[(df['marcas'] == 'Nike') & (df['current_price'] != df['normal_price'])]
promedios_nike = df_nike.groupby('month')['current_price'].mean().round(2).reset_index()

import plotly.express as px

# Calcular los valores mínimo y máximo de 'current_price'
min_price = promedios_nike['current_price'].min()
max_price = promedios_nike['current_price'].max()

# Crear una columna adicional 'color' para marcar los valores mínimos y máximos
promedios_nike['color'] = promedios_nike['current_price'].apply(lambda x: 'minimo' if x == min_price else ('maximo' if x == max_price else 'normal'))

# Crear el gráfico
fig = px.bar(promedios_nike, x='month', y='current_price', 
             title="Promedio de Precios para Nike por Mes", 
             labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)'},
             color='color',  # Usamos la columna 'color' para cambiar los colores
             color_discrete_map={'minimo': 'green', 'maximo': 'red', 'normal': 'blue'})  # Mapeo de colores


# Personalizar la leyenda
fig.update_layout(
    legend_title="Tipo de Precio",
    legend=dict(
        itemsizing='constant',  # Tamaño constante para los ítems de la leyenda
        tracegroupgap=0,  # Espaciado entre elementos en la leyenda
        font=dict(size=12)  # Tamaño de la fuente de la leyenda
    )
)

# Mostrar los nombres de los meses
fig.update_xaxes(
    tickmode='array',
    tickvals=promedios_nike['month'],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)

# Mostrar el gráfico
fig.show()

In [5]:
df_reebok = df[(df['marcas'] == 'Reebok') & (df['current_price'] != df['normal_price'])]
promedios_reebok = df_reebok.groupby('month')['current_price'].mean().round(2).reset_index()

import plotly.express as px

# Calcular los valores mínimo y máximo de 'current_price'
min_price = promedios_reebok['current_price'].min()
max_price = promedios_reebok['current_price'].max()

# Crear una columna adicional 'color' para marcar los valores mínimos y máximos
promedios_reebok['color'] = promedios_reebok['current_price'].apply(lambda x: 'minimo' if x == min_price else ('maximo' if x == max_price else 'normal'))

# Crear el gráfico
fig = px.bar(promedios_reebok, x='month', y='current_price', 
             title="Promedio de Precios para Reebok por Mes", 
             labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)'},
             color='color',  # Usamos la columna 'color' para cambiar los colores
             color_discrete_map={'minimo': 'green', 'maximo': 'red', 'normal': 'blue'})  # Mapeo de colores


# Personalizar la leyenda
fig.update_layout(
    legend_title="Tipo de Precio",
    legend=dict(
        itemsizing='constant',  # Tamaño constante para los ítems de la leyenda
        tracegroupgap=0,  # Espaciado entre elementos en la leyenda
        font=dict(size=12)  # Tamaño de la fuente de la leyenda
    )
)

# Mostrar los nombres de los meses
fig.update_xaxes(
    tickmode='array',
    tickvals=promedios_reebok['month'],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)

# Mostrar el gráfico
fig.show()

In [6]:
df_puma = df[(df['marcas'] == 'Puma') & (df['current_price'] != df['normal_price'])]
promedios_puma = df_puma.groupby('month')['current_price'].mean().round(2).reset_index()

import plotly.express as px

# Calcular los valores mínimo y máximo de 'current_price'
min_price = promedios_puma['current_price'].min()
max_price = promedios_puma['current_price'].max()

# Crear una columna adicional 'color' para marcar los valores mínimos y máximos
promedios_puma['color'] = promedios_puma['current_price'].apply(lambda x: 'minimo' if x == min_price else ('maximo' if x == max_price else 'normal'))

# Crear el gráfico
fig = px.bar(promedios_puma, x='month', y='current_price', 
             title="Promedio de Precios para Puma por Mes", 
             labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)'},
             color='color',  # Usamos la columna 'color' para cambiar los colores
             color_discrete_map={'minimo': 'green', 'maximo': 'red', 'normal': 'blue'})  # Mapeo de colores


# Personalizar la leyenda
fig.update_layout(
    legend_title="Tipo de Precio",
    legend=dict(
        itemsizing='constant',  # Tamaño constante para los ítems de la leyenda
        tracegroupgap=0,  # Espaciado entre elementos en la leyenda
        font=dict(size=12)  # Tamaño de la fuente de la leyenda
    )
)

# Mostrar los nombres de los meses
fig.update_xaxes(
    tickmode='array',
    tickvals=promedios_puma['month'],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)

# Mostrar el gráfico
fig.show()

In [7]:
df_tyr = df[(df['marcas'] == 'Tyr') & (df['current_price'] != df['normal_price'])]
promedios_tyr = df_tyr.groupby('month')['current_price'].mean().round(2).reset_index()

import plotly.express as px

# Calcular los valores mínimo y máximo de 'current_price'
min_price = promedios_tyr['current_price'].min()
max_price = promedios_tyr['current_price'].max()

# Crear una columna adicional 'color' para marcar los valores mínimos y máximos
promedios_tyr['color'] = promedios_tyr['current_price'].apply(lambda x: 'minimo' if x == min_price else ('maximo' if x == max_price else 'normal'))

# Crear el gráfico
fig = px.bar(promedios_tyr, x='month', y='current_price', 
             title="Promedio de Precios para Tyr por Mes", 
             labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)'},
             color='color',  # Usamos la columna 'color' para cambiar los colores
             color_discrete_map={'minimo': 'green', 'maximo': 'red', 'normal': 'blue'})  # Mapeo de colores


# Personalizar la leyenda
fig.update_layout(
    legend_title="Tipo de Precio",
    legend=dict(
        itemsizing='constant',  # Tamaño constante para los ítems de la leyenda
        tracegroupgap=0,  # Espaciado entre elementos en la leyenda
        font=dict(size=12)  # Tamaño de la fuente de la leyenda
    )
)

# Mostrar los nombres de los meses
fig.update_xaxes(
    tickmode='array',
    tickvals=promedios_tyr['month'],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)

# Mostrar el gráfico
fig.show()

In [8]:
df_ua = df[(df['marcas'] == 'Under Armour') & (df['current_price'] != df['normal_price'])]
promedios_ua = df_ua.groupby('month')['current_price'].mean().round(2).reset_index()

import plotly.express as px

# Calcular los valores mínimo y máximo de 'current_price'
min_price = promedios_ua['current_price'].min()
max_price = promedios_ua['current_price'].max()

# Crear una columna adicional 'color' para marcar los valores mínimos y máximos
promedios_ua['color'] = promedios_ua['current_price'].apply(lambda x: 'minimo' if x == min_price else ('maximo' if x == max_price else 'normal'))

# Crear el gráfico
fig = px.bar(promedios_ua, x='month', y='current_price', 
             title="Promedio de Precios para Under Armour por Mes", 
             labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)'},
             color='color',  # Usamos la columna 'color' para cambiar los colores
             color_discrete_map={'minimo': 'green', 'maximo': 'red', 'normal': 'blue'})  # Mapeo de colores


# Personalizar la leyenda
fig.update_layout(
    legend_title="Tipo de Precio",
    legend=dict(
        itemsizing='constant',  # Tamaño constante para los ítems de la leyenda
        tracegroupgap=0,  # Espaciado entre elementos en la leyenda
        font=dict(size=12)  # Tamaño de la fuente de la leyenda
    )
)

# Mostrar los nombres de los meses
fig.update_xaxes(
    tickmode='array',
    tickvals=promedios_ua['month'],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)

# Mostrar el gráfico
fig.show()

In [ ]:
# Filtrar los modelos deseados: 'Metcon 9', 'Tribase Reign 6' y 'Nano'
df_filtered = df[df['model'].isin(['Metcon 9', 'Tribase Reign 6', 'Nano'])]

# Agrupar por modelo y mes, y calcular el precio promedio
df_grouped = df_filtered.groupby(['month', 'model'])['current_price'].mean().reset_index()

# Crear el gráfico de líneas
fig = px.line(df_grouped, x='month', y='current_price', color='model',
              title="Comparación de Precios Promedios de Modelos Mas Importantes por Mes",
              labels={'month': 'Mes', 'current_price': 'Precio Promedio (€)', 'model': 'Modelo'},
              line_group='model')  # Asegura que cada modelo tenga su propia línea

# Mejorar la visualización del eje X para que los nombres de los meses se muestren de forma legible
fig.update_xaxes(
    tickmode='array',
    tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
)
fig.update_xaxes(tickangle=45)  # Inclinar las etiquetas del eje X para facilitar su lectura

fig.show()
